In [1]:
import logging
from irregular_object_packing.mesh.sampling import resample_pyvista_mesh
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume
from irregular_object_packing.packing.optimizer import Optimizer, default_optimizer_config 
import pyvista as pv

from irregular_object_packing.packing.optimizer_data import SimConfig

mesh_volume = 0.2
container_volume = 10

original_mesh = pv.Cube().triangulate().extract_surface()
container = pv.Cube().triangulate().extract_surface()


container = scale_to_volume(container, container_volume)
container = resample_pyvista_mesh(container, 15*4*2)
original_mesh = scale_and_center_mesh(original_mesh, mesh_volume)

settings = SimConfig(
    itn_max=100,
    n_scale_steps=9,
    r=0.3,
    final_scale=1,
    log_lvl=logging.WARNING,
    sampling_disabled=True,
    init_f=0.1,
    max_t=mesh_volume**(1 / 3) * 2,
    handle_collisions=False,
)
plotter = None
optimizer = Optimizer(original_mesh, container, settings, plotter)



In [2]:

optimizer.setup()


In [3]:

optimizer.run(Ni=1)

scale	:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration	:   0%|          | 0/100 [00:00<?, ?it/s]

Object	:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
optimizer.compute_violations()
optimizer.plotter.plot_state().show()

# optimizer.plotter.generate_gif("test.gif")

Widget(value="<iframe src='http://localhost:54590/index.html?ui=P_0x15ccc0040_0&reconnect=auto' style='width: …

In [5]:

pltr = pv.Plotter()
pltr.add_mesh(optimizer.final_cat_meshes()[0], show_edges=True, opacity=0.5)
pltr.add_mesh(optimizer.cat_mesh(0, 0), show_edges=True, opacity=0.5, color="red")

pltr.show()
# optimizer.plotter.plot_step_object_compare(5,5)

Widget(value="<iframe src='http://localhost:54590/index.html?ui=P_0x15c6982e0_1&reconnect=auto' style='width: …

In [6]:
# optimizer.plotter.plot_step_object_compare(5,5)

In [7]:
# import irregular_object_packing.cat.chordal_axis_transform as cat
# meshes_before, meshes_after, cat_meshes , _= optimizer.recreate_scene(3)
# optimizer.plotter.plot_step(4)

# meshes = meshes_before + [optimizer.container]
# tetmesh = cat.compute_cdt(meshes)

# pltr = pv.Plotter()
# pltr.add_mesh_clip_plane(tetmesh, show_edges=True, crinkle=True)
# pltr.add_mesh(cat_meshes[5], show_edges=True, color="yellow")
# pltr.add_mesh(meshes_after[5], show_edges=True, color="red")
# pltr.show()



In [8]:
optimizer = default_optimizer_config(5, '../data/mesh/')
optimizer.config.log_lvl = logging.WARNING
optimizer.setup()
optimizer.run()
optimizer.plotter.plot_state().show()

scale	:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration	:   0%|          | 0/100 [00:00<?, ?it/s]

Object	:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
optimizer.report()

In [ ]:
optimizer.fails_per_step

In [ ]:
optimizer.errors_per_step